In [45]:
import numpy as np

import mne

from mne.preprocessing import ICA
import asrpy
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, LeaveOneOut, LeavePOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot 

# Chemin vers les fichiers EEG et les positions des électrodes
raw_fnames = 'C:/recordings/Game_recordings_test/TOUT_4_second_apple/mergeddata.set'
montage_fname = 'path/to/montage_file.elc'  # Remplacez par le chemin vers votre fichier de montage

# Charger les données EEG
raw = mne.io.read_raw_eeglab(raw_fnames, preload=True)

# Charger et définir le montage
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)




Reading C:\recordings\Game_recordings_test\TOUT_4_second_apple\mergeddata.fdt
Reading 0 ... 809559  =      0.000 ...  3238.236 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


C:\Users\robinaki\AppData\Local\Temp\ipykernel_1764\1283718449.py:23: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(raw_fnames, preload=True)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [46]:
new_srate = 25
tmin = -0.5  # début de chaque époque (1,0 secondes avant le déclencheur)
tmax = 1  # fin de chaque époque (au déclencheur)
baseline = (-0.5, 0)  # baseline du début à t = 0
reject = dict(eeg=150e-3)  # seuil de rejet pour les électrodes EEG
L_H_freq = [(8,13),(13,30)]
Notchs_freq = 50
f_low = 1
raw.filter( f_low, None)
raw.notch_filter(Notchs_freq)
raw.filter(8, 30)
# Appliquer ASR pour nettoyer les artefacts
# asr = asrpy.ASR(sfreq=raw.info["sfreq"], cutoff=20)
# asr.fit(raw)
# raw = asr.transform(raw)
# ica = ICA()
# ica.fit(raw)
# raw= ica.apply(raw)
# raw.resample(new_srate)

events , event_id= mne.events_from_annotations(raw)
event_id_RL= {'right' : event_id['right'] , 'left' : event_id['left']}
epochs = mne.Epochs(raw, events, event_id_RL, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
info = epochs.info

# Calculer la matrice de covariance de bruit
noise_cov = mne.make_ad_hoc_cov(info)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------


Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---

In [47]:

# Calculer la solution inverse

# Utiliser un modèle de tête standard fourni par MNE
subjects_dir = 'C:/Users/robinaki/mne_data/MNE-sample-data/subjects'


mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)
bem = mne.read_bem_solution(
'C:/Users/robinaki/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif'
)
src = mne.setup_source_space("fsaverage", spacing="oct6", add_dist="patch", subjects_dir=subjects_dir)

fwd = mne.make_forward_solution(
    info,
    trans="fsaverage",
    src=src,
    bem=bem,
    meg=False,
    eeg=True,
    mindist=5.0,
    n_jobs=None,
    verbose=False,
)
# Calculer la solution inverse

inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8)
# stc = mne.minimum_norm.apply_inverse_raw(raw, inverse_operator, lambda2=1.0 / 9.0, method='dSPM')

# Solution of the inverse solution for evoked data (averaged data) by marker (event ID left and right)

event_id_R =  {'right' : event_id['right']}
event_id_L= {'left' : event_id['left']}
epochsR =  mne.Epochs(raw, events, event_id_R, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
epochsL =  mne.Epochs(raw, events, event_id_L, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)


stc= mne.minimum_norm.apply_inverse_epochs(
    epochs,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='sLORETA',
    pick_ori=None,
    verbose=True,
)
# stcR= mne.minimum_norm.apply_inverse_epochs(
#     epochsR,
#     inverse_operator,
#     lambda2=1.0 / 9.0,
#     method='dSPM',
#     pick_ori=None,
#     verbose=True,
# )
# stcL= mne.minimum_norm.apply_inverse_epochs(
#     epochsL,
#     inverse_operator,
#     lambda2=1.0 / 9.0,
#     method='dSPM',
#     pick_ori=None,
#     verbose=True,
# )


0 files missing from root.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects
0 files missing from bem.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage
Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Setting up the source space with the following parameters:

SUBJECTS_DIR = C:\Users\robinaki\mne_data\MNE-sample-data\subjects
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.sphere...
Setting up the triangulation for th

In [48]:
def extract_temporal_features_2(X):
    N_segmments = 4

    features = []
    # Définir les régions temporelles en secondes
    shape = np.shape(X)
    regions = [(i*shape[-1]//N_segmments ,(i+1)*shape[-1]//N_segmments) for i in range(N_segmments)  ]
    for epoch in X:  
        epoch_features = []
        for start_time, end_time in regions:
            # Calculer les indices de début et de fin pour chaque segment
            start_idx = start_time
            end_idx = end_time

            segment = epoch[:, start_idx:end_idx]

            # Calculer les statistiques pour le segment

            mean = np.mean(segment, axis=1)
            std = np.std(segment, axis=1)

            epoch_features.append(np.concatenate([mean, std]))
        features.append(np.concatenate(epoch_features))
    return np.array(features)
def extract_temporal_features(X):
    """
    Extact mean, std and higher order moments
    """
    features = []
    i=0
    tott = len(X)
    for epoch in X:
        mean = np.mean(epoch, axis=-1)
        std = np.std(epoch, axis=-1)
        features.append(np.concatenate([mean,std]))#mean
        i+=1
    print(np.shape(features))
    return (np.array(features))

In [49]:

X_src = []
y= epochs.events[:, -1] == event_id['right']
for epoch in stc:
    X_src += [epoch.data]
print('extracting features....')
X = extract_temporal_features(X_src)
# SVC(kernel='linear')
print('extracting features finished')
    

extracting features....
extracting features finished


In [50]:

pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

cv = StratifiedKFold(20, shuffle=True)
# cv = LeavePOut(5)


scores = cross_val_score(pipeline,X , y, cv = cv)
print(f"Cross validation scores: {scores}")
print(f"Mean: {scores.mean()}")



Cross validation scores: [0.75       0.69444444 0.72222222 0.69444444 0.63888889 0.66666667
 0.77142857 0.71428571 0.77142857 0.62857143 0.57142857 0.71428571
 0.62857143 0.68571429 0.8        0.74285714 0.68571429 0.74285714
 0.71428571 0.6       ]
Mean: 0.6969047619047618


In [51]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer LASSO pour la sélection de caractéristiques
lasso = Lasso(alpha=0.01)  # Vous pouvez ajuster alpha pour la régularisation
lasso.fit(X_train, y_train)

# Sélectionner les caractéristiques importantes
selector = SelectFromModel(lasso, prefit=True)
X_train_reduced = selector.transform(X_train)
X_test_reduced = selector.transform(X_test)

# Entraîner un modèle de classification avec GradientBoostingClassifier
model = RandomForestClassifier()
model.fit(X_train_reduced, y_train)

# Prédire et évaluer le modèle
y_pred = model.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Nombre de caractéristiques sélectionnées: {X_train_reduced.shape[1]}')

# Optionnel : Afficher les indices des caractéristiques sélectionnées
selected_features = selector.get_support(indices=True)
print(f'Indices des caractéristiques sélectionnées: {selected_features}')

c:\Users\robinaki\AppData\Local\mne-python\1.7.1_0\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.559e+00, tolerance: 1.410e-02
  model = cd_fast.enet_coordinate_descent(


Accuracy: 0.70
Nombre de caractéristiques sélectionnées: 126
Indices des caractéristiques sélectionnées: [   29    81   242   321   443   632   749  1080  1220  1836  1867  1992
  2100  2120  3280  3281  3699  4209  4486  4697  4850  5030  5263  5475
  5903  7362  7396  7566  7747  8246  8518  8627  8633  8634  9110  9344
 10006 10063 10163 10590 10911 11713 11974 12025 12690 13468 13767 14949
 14960 15103 15442 15755 17010 17042 18484 18513 18787 19717 19906 19907
 21786 22943 23825 24384 24633 25280 25767 26941 27563 27719 27907 28237
 28798 28887 31440 31967 33121 33375 33548 34004 34068 34087 34222 34485
 34776 34777 34954 34963 36322 36595 37359 37537 37900 39119 39448 40041
 40092 40632 41116 48898 49768 50089 52171 52312 53673 53870 53876 54026
 54853 55377 55925 56640 57088 58880 58912 59106 59543 60077 60201 60678
 60712 60949 61889 62595 63427 64390]


In [68]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
def Scores_feature_select(X_train, X_test, y_train, y_test ,model_feature_select=PCA(n_components=50),model = SVC()):
    """ Input: Train and test data for training selected classifier with selected features selection
        Output : Accuracy for the test set """

    from sklearn.metrics import accuracy_score
    model_select = model_feature_select
    X_train_select = model_select.fit_transform(X_train)
    X_test_select = model_select.transform(X_test)

    # Entraîner un modèle de classification avec GradientBoostingClassifier
    model.fit(X_train_select, y_train)

    # Prédire et évaluer le modèle
    y_pred = model.predict(X_test_select)
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Accuracy: {accuracy:.2f}')
    print(f'Variance expliquée par les {n_components} composantes principales: {np.sum(pca.explained_variance_ratio_):.2f}')

    return accuracy
def Sores_RandomForest(X_train, X_test, y_train, y_test,n_estimators = 100 , model = SVC()):
    selector = SelectFromModel(RandomForestClassifier(n_estimators=n_estimators))
    selector.fit(X_train, y_train)
    X_train_reduced = selector.transform(X_train)
    X_test_reduced = selector.transform(X_test)

    model.fit(X_train_reduced, y_train)
    y_pred = model.predict(X_test_reduced)

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')
    return accuracy 

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def 
# Sélection de caractéristiques avec RandomForest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100))
selector.fit(X_train, y_train)
X_train_reduced = selector.transform(X_train)
X_test_reduced = selector.transform(X_test)

# Classification avec GradientBoostingClassifier
selector = SelectFromModel(RandomForestClassifier(n_estimators=100))
model = SVC()
model.fit(X_train_reduced, y_train)
y_pred = model.predict(X_test_reduced)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.70


In [53]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer RFE pour la sélection de caractéristiques avec RandomForest comme modèle de base
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(estimator=base_model, n_features_to_select=50, step=100)  # Vous pouvez ajuster n_features_to_select
rfe.fit(X_train, y_train)

# Réduire les données aux caractéristiques sélectionnées
X_train_reduced = rfe.transform(X_train)
X_test_reduced = rfe.transform(X_test)

# Entraîner un modèle de classification avec GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

# Prédire et évaluer le modèle
y_pred = model.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Nombre de caractéristiques sélectionnées: {X_train_reduced.shape[1]}')

# Optionnel : Afficher les indices des caractéristiques sélectionnées
selected_features = rfe.get_support(indices=True)
print(f'Indices des caractéristiques sélectionnées: {selected_features}')


Accuracy: 0.63
Nombre de caractéristiques sélectionnées: 50
Indices des caractéristiques sélectionnées: [  693  1275  1510  1610  1802  2420  2548  2555  2773  3357  3461  3556
  4177  4229  4321  4627  4777  4921  4986  5263  5354 60758 61014 61027
 61152 61157 61172 61173 61250 61251 61298 61324 61337 61385 61397 61441
 62072 62147 62421 62749 62773 63020 63410 63786 64184 64780 64783 64932
 65258 65280]


In [71]:
Scores_PCA(X_train, X_test, y_train, y_test ,n_components=50,model = LogisticRegression(max_iter =1000))

Accuracy: 0.73
Variance expliquée par les 50 composantes principales: 1.00


0.7323943661971831

In [66]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skrebate import ReliefF

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer ReliefF pour la sélection de caractéristiques
relieff = ReliefF(n_neighbors=100, n_features_to_select=50)  # Vous pouvez ajuster ces paramètres
relieff.fit(X_train, y_train)

# Réduire les données aux caractéristiques sélectionnées
X_train_reduced = relieff.transform(X_train)
X_test_reduced = relieff.transform(X_test)

# Entraîner un modèle de classification avec GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

# Prédire et évaluer le modèle
y_pred = model.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Nombre de caractéristiques sélectionnées: {X_train_reduced.shape[1]}')

# Optionnel : Afficher les indices des caractéristiques sélectionnées
selected_features = relieff.top_features_
print(f'Indices des caractéristiques sélectionnées: {selected_features[:50]}')


KeyboardInterrupt: 